<a href="https://colab.research.google.com/github/Jane28560820/2025_Agent_club/blob/main/R1_HW_Jane.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# baseline

將`關鍵字`比對換成`向量相似度`比對。

> 請將目前使用關鍵字比對的 route_by_query，改為使用向量相似度進行分類，並設一個合理的相似度門檻，根據檢索結果的分數判斷是否走 RAG 流程。  
例如用向量相似度及自訂 threshold 決定要不要分到 retriever。

> Hint：similarity_search_with_score(...)  
可參考去年的讀書會 R4：向量資料庫的基本操作

In [ ]:
!pip install -q langchain langgraph transformers bitsandbytes langchain-huggingface langchain-community chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 41.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.9/18.9 MB 44.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 80.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.2/44.2 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 3.8 MB/s eta 0:00:0

In [ ]:
from langchain_core.documents import Document
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings

# RAG需要的文件
docs_text = """
火影代數	姓名	師傅	徒弟
初代	千手柱間	無明確記載	猿飛日斬、水戶門炎、轉寢小春
二代	千手扉間	千手柱間（兄長）	猿飛日斬、志村團藏、宇智波鏡等
三代	猿飛日斬	千手柱間、千手扉間	自來也、大蛇丸、千手綱手（傳說三忍）
四代	波風湊	自來也	旗木卡卡西、宇智波帶土、野原琳
五代	千手綱手	猿飛日斬	春野櫻、志乃等（主要為春野櫻）
六代	旗木卡卡西	波風湊	漩渦鳴人、宇智波佐助、春野櫻（第七班）
七代	漩渦鳴人	自來也、旗木卡卡西	木葉丸等（主要為木葉丸）
"""

# RAG 系統中每一段文本都需要封裝成 Document
docs = [Document(page_content=txt.strip()) for txt in docs_text.strip().split("\n\n")]

# chromadb 預設使用的大型語言模型為 "all-MiniLM-L6-v2"，由於該大型語言模型不支持中文，所以將模型替換為 "infgrad/stella-base-zh-v3-1792d"，並對 embedding 進行量化

# 建立中文 embedding 模型
embedding_model = HuggingFaceEmbeddings(
    model_name="infgrad/stella-base-zh-v3-1792d",
    encode_kwargs={"normalize_embeddings": True}
)

# 建立 Chroma 向量資料庫並儲存
persist_path = "document_store"
collection_name = "naruto_collection"
vectorstore = Chroma.from_documents(
    documents=docs,
    embedding=embedding_model,
    persist_directory=persist_path,
    collection_name=collection_name
)

<ipython-input-2-d3bde430de66>:23: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/311 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/32.1k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/51.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/533 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/408M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/5.51M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/121 [00:00<?, ?B/s]

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, BitsAndBytesConfig
from langchain_huggingface import HuggingFacePipeline

# 1. 生成回應模型載入

# 使用 4-bit 量化模型
model_id = "MediaTek-Research/Breeze-7B-Instruct-v1_0"

# 設定量化參數
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    llm_int8_threshold=6.0,
)

# 載入 tokenizer 與 4-bit 模型
tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    quantization_config=quant_config,
    trust_remote_code=True
)

# 建立 text generation pipeline
generator = pipeline(
    task="text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    do_sample=True,
    temperature=0.4,
    return_full_text=False # 僅返回生成的回應內容
)

tokenizer_config.json:   0%|          | 0.00/2.33k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/911k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.79M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

...

model-00004-of-00004.safetensors:   0%|          | 0.00/508M [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
# 2. 定義狀態（State）
from typing_extensions import TypedDict, List

# 定義 LangGraph 的 State 結構
class RAGState(TypedDict):
    query: str            #使用者問題
    docs: List[Document]  #檢索到的文件
    answer: str   #模型答案


# 3. 定義節點（Node）

# 用來 retrive 歷代火影資料的節點
def retrieve_node(state: RAGState) -> RAGState:
    query = state["query"]
    # similarity_search 距離越小越相似
    docs = vectorstore.similarity_search(query, k=3)
    return {"query": query, "docs": docs, "answer": ""}

# 用來 retrive 後生成的節點
def generate_node(state: RAGState) -> RAGState:
    query, docs = state["query"], state["docs"]
    context = "\n".join([d.page_content for d in docs])
    prompt = (
        f"你是一個知識型助手，請根據以下內容回答問題：\n\n"
        f"內容：{context}\n\n"
        f"問題：{query}\n\n回答："
    )
    output = generator(prompt, max_new_tokens=200)[0]["generated_text"]
    return {"query": query, "docs": docs, "answer": output}

# 直接生成的節點
def direct_generate_node(state: RAGState) -> RAGState:
    query = state["query"]
    prompt = f"請回答以下問題：{query}\n\n回答："
    output = generator(prompt, max_new_tokens=200)[0]["generated_text"]
    return {"query": query, "docs": [], "answer": output}

# 定義 Route Node 條件分支（決定走哪條路）
# def route_by_query(state):
#     query = state["query"]
#     threshold = 0.8  # 數值越小代表越相似

#     # 使用語意檢索（距離越小越相似）
#     results_with_score = vectorstore.similarity_search_with_score(query, k=1)

#     if results_with_score:
#         _, score = results_with_score[0]  # 取出最相似的一筆資料的分數
#         print(f"相似度分數：{score:.4f}")
#         if score < threshold:
#             choice = "naruto"
#             print(f"跑到 → {choice}")
#             return choice
#         else:
#             choice = "general"
#             print(f"跑到 → {choice}")
#             return choice


from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def route_by_query(state):
    query = state["query"]
    query_embedding = embedding_model.embed_query(query)  # 取得 query 的向量

    # 抓出向量庫中最接近的 doc 和其向量
    results_with_score = vectorstore.similarity_search_with_score(query, k=1)

    if results_with_score:
        doc, _ = results_with_score[0]
        doc_embedding = embedding_model.embed_query(doc.page_content)  # 取得文件的向量

        # 計算 cosine similarity
        sim = cosine_similarity(
            np.array(query_embedding).reshape(1, -1),
            np.array(doc_embedding).reshape(1, -1)
        )[0][0]

        print(f"Cosine similarity：{sim:.4f}")

        if sim > 0.7:  # 相似度越高越相似，這裡是你可以調的門檻
            choice = "naruto"
            print(f"跑到 → {choice}")
            return choice

        else:
            choice = "general"
            print(f"跑到 → {choice}")
            return choice

In [ ]:
# 4. 建立 LangGraph 流程圖（StateGraph）
from langchain_core.runnables import RunnableLambda
from langgraph.graph import StateGraph, END

# 建立 LangGraph 流程圖
graph_builder = StateGraph(RAGState)

graph_builder.set_entry_point("condition")
graph_builder.add_node("condition", RunnableLambda(lambda x: x))  # 進來就分流，不改變內容
graph_builder.add_node("retriever", RunnableLambda(retrieve_node))
graph_builder.add_node("generator", RunnableLambda(generate_node))
graph_builder.add_node("direct_generator", RunnableLambda(direct_generate_node))

# 設定條件分流
graph_builder.add_conditional_edges(
    source="condition",
    path=RunnableLambda(route_by_query),
    path_map={
        "naruto": "retriever",
        "general": "direct_generator",
    }
)

# 接下來的正常連接
graph_builder.add_edge("retriever", "generator")
graph_builder.add_edge("generator", END)
graph_builder.add_edge("direct_generator", END)

# 編譯 Graph
graph = graph_builder.compile()

In [ ]:
# 5. 建立 RAG 結果

print("開始對話吧（輸入 q 結束）")

while True:
    user_input = input("使用者: ")
    if user_input.strip().lower() in ["q", "quit", "exit"]:
        print("掰啦！")
        break

    # 初始化 State（RAGState）
    init_state: RAGState = {
        "query": user_input,
        "docs": [],
        "answer": ""
    }

    result = graph.invoke(init_state) # 執行 LangGraph 流程圖
    raw_output = result["answer"]

    answer_text = raw_output.split("回答：")[-1].strip()
    print("回答：", answer_text)
    print("===" * 20, "\n")

開始對話吧（輸入 q 結束）
使用者: 誰是第四代火影?
Cosine similarity：0.7023
跑到 → naruto
回答： 第四代火影是波風湊。

使用者: 第四代火影的師傅是誰?
Cosine similarity：0.7398
跑到 → naruto
回答： 第四代火影的師傅是自來也。

使用者: 第四代火影的徒弟有哪些人?
Cosine similarity：0.7476
跑到 → naruto
回答： 第四代火影的徒弟有旗木卡卡西、宇智波帶土、野原琳。

使用者: 相對論是誰發明的?
Cosine similarity：0.3545
跑到 → general
回答： 相對論是由愛因斯坦（Albert Einstein）在 1905 年所發明的。

使用者: quit
掰啦！


# advance

改成能支援多輪問答（Multi-turn RAG），並能根據前面的query判斷問題。

> 請將 RAGState 加入 history 欄位，並在生成回答時，將歷史對話與當前問題一起組成 prompt。

> Hint：
```
class MultiTurnRAGState(TypedDict):  
    history: List[str]  
    query: str  
    docs: List[Document]  
    answer: str
```



In [ ]:
# 2. 定義狀態（State）
from typing_extensions import TypedDict, List

# 定義 LangGraph 的 State 結構
class MultiTurnRAGState(TypedDict):
    history: List[str]
    query: str
    docs: List[Document]
    answer: str

# 3. 定義節點（Node）

# 用來 retrive 歷代火影資料的節點
def retrieve_node(state: RAGState) -> RAGState:
    query = state["query"]
    # similarity_search 距離越小越相似
    docs = vectorstore.similarity_search(query, k=3)
    return {"query": query, "docs": docs, "answer": ""}

# 加入history後 用來 retrive 後生成的節點
def generate_node(state: MultiTurnRAGState) -> MultiTurnRAGState:
    query, docs, history = state["query"], state["docs"], state["history"]
    context = "\n".join([d.page_content for d in docs])

    # 將歷史對話串成一段對話文本
    history_text = "\n".join([f"對話{i+1}：{msg}" for i, msg in enumerate(history)])

    prompt = (
        f"你是一個知識型助手，請根據歷史對話與下列內容回答使用者的問題。\n\n"
        f"{history_text}\n\n"
        f"文件內容：\n{context}\n\n"
        f"問題：{query}\n\n回答："
    )
    output = generator(prompt, max_new_tokens=200)[0]["generated_text"]

    # 更新歷史對話（加入這輪問題與回答）
    new_history = history + [f"使用者：{query}", f"AI：{output}"]
    return {"query": query, "docs": docs, "answer": output, "history": new_history}



# 加入history後的 直接生成的節點
def direct_generate_node(state: MultiTurnRAGState) -> MultiTurnRAGState:
    query, history = state["query"], state["history"]
    history_text = "\n".join([f"對話{i+1}：{msg}" for i, msg in enumerate(history)])

    prompt = (
        f"你是一個知識型助手，請根據歷史對話回答使用者的問題。\n\n"
        f"{history_text}\n\n"
        f"請回答以下問題：{query}\n\n回答"
    )
    output = generator(prompt, max_new_tokens=200)[0]["generated_text"]

    new_history = history + [f"使用者：{query}", f"AI：{output}"]
    return {"query": query, "docs": [], "answer": output, "history": new_history}


from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def route_by_query(state):
    query = state["query"]
    history = state.get("history", [])

    # 組合語意查詢內容（以最近兩句為上下文）
    if history:
        recent_history = "\n".join(history[-2:])  # 最後一輪對話（使用者 + AI）
        full_query = f"{recent_history}\n{query}"
    else:
        full_query = query

    query_embedding = embedding_model.embed_query(full_query)

    results_with_score = vectorstore.similarity_search_with_score(query, k=1)
    if results_with_score:
        doc, _ = results_with_score[0]
        doc_embedding = embedding_model.embed_query(doc.page_content)

        sim = cosine_similarity(
            np.array(query_embedding).reshape(1, -1),
            np.array(doc_embedding).reshape(1, -1)
        )[0][0]

        print(f"Cosine similarity：{sim:.4f}")
        if sim > 0.6:
            choice = "naruto"
            print(f"跑到 → {choice}")
            return choice
        else:
            choice = "general"
            print(f"跑到 → {choice}")
            return choice

In [ ]:
# 4. 建立 LangGraph 流程圖（StateGraph）
from langchain_core.runnables import RunnableLambda
from langgraph.graph import StateGraph, END

# 改成 MultiTurnRAGState 建立 LangGraph 流程圖
graph_builder = StateGraph(MultiTurnRAGState)

graph_builder.set_entry_point("condition")
graph_builder.add_node("condition", RunnableLambda(lambda x: x))  # 進來就分流，不改變內容
graph_builder.add_node("retriever", RunnableLambda(retrieve_node))
graph_builder.add_node("generator", RunnableLambda(generate_node))
graph_builder.add_node("direct_generator", RunnableLambda(direct_generate_node))

# 設定條件分流
graph_builder.add_conditional_edges(
    source="condition",
    path=RunnableLambda(route_by_query),
    path_map={
        "naruto": "retriever",
        "general": "direct_generator",
    }
)

# 接下來的正常連接
graph_builder.add_edge("retriever", "generator")
graph_builder.add_edge("generator", END)
graph_builder.add_edge("direct_generator", END)

# 編譯 Graph
graph = graph_builder.compile()

In [ ]:
global_history: List[str] = []

print("開始對話吧（輸入 q 結束）")
while True:
    user_input = input("使用者: ")
    if user_input.strip().lower() in ["q", "quit", "exit"]:
        print("掰啦！")
        break

    state = {
    "query": user_input,
    "docs": [],
    "answer": "",
    "history": global_history
    }

    result = graph.invoke(state)

    answer = result["answer"].split("回答：")[-1].strip()
    print("history:",global_history)
    print("AI 助理:", answer)
    print("===" * 60, "\n")

    global_history = result["history"]

開始對話吧（輸入 q 結束）
使用者: 第四代火影是誰?
Cosine similarity：0.6912
跑到 → naruto
history: []
AI 助理: 第四代火影是波風湊。

使用者: 他的師父是誰?
Cosine similarity：0.7134
跑到 → naruto
history: ['使用者：第四代火影是誰?', 'AI：第四代火影是波風湊。']
AI 助理: 他的師父是自來也。

使用者: 他的徒弟有哪些人?
Cosine similarity：0.6509
跑到 → naruto
history: ['使用者：第四代火影是誰?', 'AI：第四代火影是波風湊。', '使用者：他的師父是誰?', 'AI：他的師父是自來也。']
AI 助理: 他的徒弟有旗木卡卡西、宇智波佐助、春野櫻（第七班）。

使用者: 相對論是他發明的嗎?
Cosine similarity：0.6484
跑到 → naruto
history: ['使用者：第四代火影是誰?', 'AI：第四代火影是波風湊。', '使用者：他的師父是誰?', 'AI：他的師父是自來也。', '使用者：他的徒弟有哪些人?', 'AI：他的徒弟有旗木卡卡西、宇智波佐助、春野櫻（第七班）。']
AI 助理: 相對論並非是第四代火影波風湊所發明的。相對論是一種理論，主要由愛因斯坦在1905年提出，它描述了重力和加速度等現象。

使用者: quit
掰啦！
